In [1]:
import numpy as np
import pandas as pd

In [2]:
pop = pd.read_csv('msa_pop.csv', encoding = "ISO-8859-1")

In [3]:
zip_ = pd.read_csv('zip_to_msa.csv',dtype='str').drop(columns=['Unnamed: 4'])

In [4]:
zip_.msa_name = zip_.msa_name.replace('nan', np.nan)
pop.msa_name = pop.msa_name.replace('nan', np.nan)

In [5]:
zip_.msa_name.dropna(inplace=True)
pop.msa_name.dropna(inplace=True)

In [6]:
def clean(x):
    return ('').join(((' ').join(x.split("-"))).split(","))

In [7]:
pop.msa_name = pop.msa_name.apply(lambda x: clean(x))

In [8]:
zip_.msa_name = zip_.msa_name.apply(lambda x: clean(x))

In [9]:
zip_.msa_name

0        Boston Cambridge Quincy MA NH Metro Area
1        Boston Cambridge Quincy MA NH Metro Area
2        Boston Cambridge Quincy MA NH Metro Area
3        Boston Cambridge Quincy MA NH Metro Area
4        Boston Cambridge Quincy MA NH Metro Area
                           ...                   
43503                     AK NONMETROPOLITAN AREA
43504                     AK NONMETROPOLITAN AREA
43505                     AK NONMETROPOLITAN AREA
43506                     AK NONMETROPOLITAN AREA
43507                     AK NONMETROPOLITAN AREA
Name: msa_name, Length: 43508, dtype: object

In [10]:
from fuzzywuzzy import fuzz
name = 'temp'
temp = 0
d = {}
for msa_zip in list(set(zip_.msa_name)):
    for msa_pop in list(set(pop.msa_name)):
        ratio = fuzz.token_sort_ratio(msa_zip, msa_pop)
        if ratio > temp:
            temp = ratio
            name = msa_pop
    if temp >= 70:
        d[msa_zip] = name
        temp = 0

In [11]:
d['Boston Cambridge Quincy MA NH Metro Area']

'Boston Cambridge Newton MA NH Metro Area'

In [12]:
def try_fuzz(x):
    try:
        return d[x]
    except:
        return np.nan

In [13]:
zip_['msa_name_fuzz'] = zip_.msa_name.apply(lambda x: try_fuzz(x))

In [14]:
merged_msa = pd.merge(left=zip_ , right=pop, how='left', left_on='msa_name_fuzz', right_on='msa_name')

In [15]:
merged_msa.dropna(inplace=True)

In [16]:
merged_msa.columns

Index(['zip_code', 'state', 'msa_num', 'msa_name_x', 'msa_name_fuzz',
       'msa_name_y', 'pop_2018'],
      dtype='object')

In [17]:
merged_msa[['msa_name_fuzz','msa_name_x']]

,msa_name_fuzz,msa_name_x
0,Boston Cambridge Newton MA NH Metro Area,Boston Cambridge Quincy MA NH Metro Area
1,Boston Cambridge Newton MA NH Metro Area,Boston Cambridge Quincy MA NH Metro Area
2,Boston Cambridge Newton MA NH Metro Area,Boston Cambridge Quincy MA NH Metro Area
3,Boston Cambridge Newton MA NH Metro Area,Boston Cambridge Quincy MA NH Metro Area
4,Boston Cambridge Newton MA NH Metro Area,Boston Cambridge Quincy MA NH Metro Area
...,...,...
43404,Fairbanks AK Metro Area,Fairbanks AK Metro Area
43405,Fairbanks AK Metro Area,Fairbanks AK Metro Area
43411,Fairbanks AK Metro Area,Fairbanks AK Metro Area
43459,Fairbanks AK Metro Area,Fairbanks AK Metro Area


In [20]:
merged_msa.to_csv('zipcode_population.csv')